# PySpark YouTube Course - Lesson 02

This notebook contains all the materials and notes from the PySpark Course: 'PySpark - Aula 02 - Window Functions - Português - Hands On' by DataDev Academy, available on YouTube. You can find the video [here](https://www.youtube.com/watch?v=MG-QBpEXvjg).

## Summary

- [PySpark YouTube Course - Lesson 02](#pyspark-youtube-course---lesson-02)
  - [Summary](#summary)
  - [Importing PySpark](#importing-pyspark)
  - [Starting PySpark Session](#starting-pyspark-session)
  - [Reading a File](#reading-a-file)
  - [Displaying the DataFrame](#displaying-the-dataframe)
  - [PySpark 01 Changes](#pyspark-01-changes)
  - [Creating Backup](#creating-backup)
  - [Window Ranking Functions](#window-ranking-functions)
    - [Window Function 1 - Number of Rows: `row_number()`](#window-function-1---number-of-rows-row_number)
    - [Window Function 2 - Ranking 1: `rank()`](#window-function-2---ranking-1-rank)
    - [Window Function 3 - Ranking 2: `dense_rank()`](#window-function-3---ranking-2-dense_rank)
    - [Window Function 4 - Percentage Ranking: `percent_rank()`](#window-function-4---percentage-ranking-percent_rank)
    - [Window Function 5 - Division into 'N' Parts: `ntile()`](#window-function-5---division-into-n-parts-ntile)
  - [Window Analytic Functions](#window-analytic-functions)
    - [Window Function 6 - LAG / Offset: `lag()`](#window-function-6---lag--offset-lag)
    - [Window Function 7 - Lead / Offset: `lead()`](#window-function-7---lead--offset-lead)
  - [Aggregations](#aggregations)
    - [GroupBy + AGG 1](#groupby--agg-1)
    - [GroupBy + AGG 2](#groupby--agg-2)
  - [Where (Filter)](#where-filter)
  - [Describe](#describe)
  - [Aggregation Function Using Window Function](#aggregation-function-using-window-function)

## Importing PySpark

In [51]:
# Importing PySpark modules
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window  # Importing window functions

## Starting PySpark Session

In [52]:
# Creating a Spark session
# - master: defines the cluster manager, 'local' means running on a local machine
# - appName: sets the name of your application
spark = SparkSession.builder \
    .master("local") \
    .appName("pyspark-02") \
    .getOrCreate()

## Reading a File

In [53]:
# Reading a CSV file into a DataFrame
# - path: specifies the location of the file
# - header: indicates if the file contains a header row
# - inferSchema: automatically infers the data types of the columns
df = spark.read.csv("./data/wc2018-players.csv", header=True, inferSchema=True)

## Displaying the DataFrame

In [54]:
# - show(5): shows the first 5 rows of the DataFrame
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



## PySpark 01 Changes

In [55]:
# Renaming columns in the DataFrame
# - withColumnRenamed("oldName", "newName"): renames a column from oldName to newName
df = df.withColumnRenamed("Pos.", "Position") \
       .withColumnRenamed("#", "Number")

In [56]:
# Extracting and converting components from the "Birth Date" column
df = df.withColumn("Day", split(col("Birth Date"), "\\.").getItem(0).cast(IntegerType())) \
       .withColumn("Month", split(col("Birth Date"), "\\.").getItem(1).cast(IntegerType())) \
       .withColumn("Year", split(col("Birth Date"), "\\.").getItem(2).cast(IntegerType()))

In [57]:
# Creating a new "Birth" column by concatenating "Year", "Month", and "Day" columns
df = df.withColumn("Birth", concat_ws("-", col("Year"), col("Month"), col("Day")).cast(DateType()))

# Displaying the first 5 rows of the DataFrame to verify the changes
df.show(5)

+---------+------+--------+------------------+----------+----------+--------------------+------+------+---+-----+----+----------+
|     Team|Number|Position| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|Day|Month|Year|     Birth|
+---------+------+--------+------------------+----------+----------+--------------------+------+------+---+-----+----+----------+
|Argentina|     3|      DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65| 31|    8|1992|1992-08-31|
|Argentina|    22|      MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65| 21|    1|1996|1996-01-21|
|Argentina|    15|      MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66| 15|    2|1993|1993-02-15|
|Argentina|    18|      DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69| 13|    7|1990|1990-07-13|
|Argentina|    10|      FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)| 

In [58]:
# Printing the schema of the DataFrame to show the structure and data types of each column
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- Number: integer (nullable = true)
 |-- Position: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Birth: date (nullable = true)



In [59]:
# Dropping the "Birth Date" column from the DataFrame
df = df.drop(col("Birth Date"))

# Displaying the first 5 rows of the DataFrame to verify the changes
df.show(5)

+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|                Club|Height|Weight|Day|Month|Year|     Birth|
+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+
|Argentina|     3|      DF|TAGLIAFICO Nicolas|TAGLIAFICO|      AFC Ajax (NED)|   169|    65| 31|    8|1992|1992-08-31|
|Argentina|    22|      MF|    PAVON Cristian|     PAVÓN|CA Boca Juniors (...|   169|    65| 21|    1|1996|1996-01-21|
|Argentina|    15|      MF|    LANZINI Manuel|   LANZINI|West Ham United F...|   167|    66| 15|    2|1993|1993-02-15|
|Argentina|    18|      DF|    SALVIO Eduardo|    SALVIO|    SL Benfica (POR)|   167|    69| 13|    7|1990|1990-07-13|
|Argentina|    10|      FW|      MESSI Lionel|     MESSI|  FC Barcelona (ESP)|   170|    72| 24|    6|1987|1987-06-24|
+---------+------+--------+------------------+--

## Creating Backup

In [60]:
df2 = df

## Window Ranking Functions

- **Window Function 1 - Number of Rows: `row_number()`**
  - Assigns a unique sequential number to each row within a window partition.

- **Window Function 2 - Ranking 1: `rank()`**
  - Assigns ranks to rows within a partition, with gaps in rank values for ties.

- **Window Function 3 - Ranking 2: `dense_rank()`**
  - Assigns ranks to rows within a partition, without gaps in rank values for ties.

- **Window Function 4 - Percentage Ranking: `percent_rank()`**
  - Calculates the relative rank of each row as a percentage within a partition.

- **Window Function 5 - Division into 'N' Parts: `ntile()`**
  - Distributes rows into a specified number of roughly equal-sized buckets or tiles.

### Window Function 1 - Number of Rows: `row_number()`

In [61]:
# Define the window specification
# - partitionBy(col("Team")): partition the data by the "Team" column, so each team will have its own separate window
# - orderBy(desc(col("Height"))): order the rows within each partition by the "Height" column in descending order
num_line = Window.partitionBy(col("Team")).orderBy(desc(col("Height")))

# Add a new column "Line Number" to the DataFrame
# - row_number(): generates a unique sequential number for each row within the specified window
# - over(num_line): applies the window specification defined above to determine the row number
# This means that within each team, rows will be numbered starting from 1 for the highest height
df.withColumn("Line Number", row_number().over(num_line)).show(50)


+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+-----------+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|                Club|Height|Weight|Day|Month|Year|     Birth|Line Number|
+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+-----------+
|Argentina|     6|      DF|    FAZIO Federico|     FAZIO|       AS Roma (ITA)|   199|    85| 17|    3|1987|1987-03-17|          1|
|Argentina|     1|      GK|     GUZMAN Nahuel|    GUZMÁN|   Tigres UANL (MEX)|   192|    90| 10|    2|1986|1986-02-10|          2|
|Argentina|    16|      DF|       ROJO Marcos|      ROJO|Manchester United...|   189|    82| 20|    3|1990|1990-03-20|          3|
|Argentina|    12|      GK|     ARMANI Franco|    ARMANI|CA River Plate (ARG)|   189|    85| 16|   10|1986|1986-10-16|          4|
|Argentina|    23|      GK|CABALLERO Wilfredo| CABALLERO|    Chelsea FC (ENG)|   18

### Window Function 2 - Ranking 1: `rank()`

In [62]:
# Define the window specification
# - partitionBy(col("Team")): partition the data by the "Team" column, so each team will have its own separate window
# - orderBy(desc(col("Height"))): order the rows within each partition by the "Height" column in descending order
rank1 = Window.partitionBy(col("Team")).orderBy(desc(col("Height")))

# Add a new column "Rank" to the DataFrame
# - rank(): assigns a rank to each row within the specified window, with the highest height receiving rank 1
# - over(rank1): applies the window specification defined above to compute the rank
# This means that within each team, rows will be ranked starting from 1 for the highest height
df.withColumn("Rank", rank().over(rank1)).show(50)

+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+----+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|                Club|Height|Weight|Day|Month|Year|     Birth|Rank|
+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+----+
|Argentina|     6|      DF|    FAZIO Federico|     FAZIO|       AS Roma (ITA)|   199|    85| 17|    3|1987|1987-03-17|   1|
|Argentina|     1|      GK|     GUZMAN Nahuel|    GUZMÁN|   Tigres UANL (MEX)|   192|    90| 10|    2|1986|1986-02-10|   2|
|Argentina|    16|      DF|       ROJO Marcos|      ROJO|Manchester United...|   189|    82| 20|    3|1990|1990-03-20|   3|
|Argentina|    12|      GK|     ARMANI Franco|    ARMANI|CA River Plate (ARG)|   189|    85| 16|   10|1986|1986-10-16|   3|
|Argentina|    23|      GK|CABALLERO Wilfredo| CABALLERO|    Chelsea FC (ENG)|   186|    80| 28|    9|1981|1981-09-28|   5|
|Argenti

### Window Function 3 - Ranking 2: `dense_rank()`

In [63]:
# Define the window specification
# - partitionBy(col("Team")): partition the data by the "Team" column, so each team will have its own separate window
# - orderBy(desc(col("Height"))): order the rows within each partition by the "Height" column in descending order
rank2 = Window.partitionBy(col("Team")).orderBy(desc(col("Height")))

# Add a new column "Rank 2" to the DataFrame
# - dense_rank(): assigns a dense rank to each row within the specified window, with the highest height receiving rank 1
# - over(rank2): applies the window specification defined above to compute the dense rank
#   Rows with the same height receive the same rank, and there are no gaps in the rank sequence
df.withColumn("Rank 2", dense_rank().over(rank2)).show(50)

+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+------+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|                Club|Height|Weight|Day|Month|Year|     Birth|Rank 2|
+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+------+
|Argentina|     6|      DF|    FAZIO Federico|     FAZIO|       AS Roma (ITA)|   199|    85| 17|    3|1987|1987-03-17|     1|
|Argentina|     1|      GK|     GUZMAN Nahuel|    GUZMÁN|   Tigres UANL (MEX)|   192|    90| 10|    2|1986|1986-02-10|     2|
|Argentina|    16|      DF|       ROJO Marcos|      ROJO|Manchester United...|   189|    82| 20|    3|1990|1990-03-20|     3|
|Argentina|    12|      GK|     ARMANI Franco|    ARMANI|CA River Plate (ARG)|   189|    85| 16|   10|1986|1986-10-16|     3|
|Argentina|    23|      GK|CABALLERO Wilfredo| CABALLERO|    Chelsea FC (ENG)|   186|    80| 28|    9|1981|1981-09-28|

### Window Function 4 - Percentage Ranking: `percent_rank()`

In [64]:
# Define the window specification
# - partitionBy(col("Team")): partition the data by the "Team" column, so each team will have its own separate window
# - orderBy(desc(col("Height"))): order the rows within each partition by the "Height" column in descending order
percent = Window.partitionBy(col("Team")).orderBy(desc(col("Height")))

# Add a new column "%" to the DataFrame
# - percent_rank(): calculates the percentage rank for each row within the specified window
# - over(percent): applies the window specification defined above to compute the percentage rank
# This means that within each team, rows will have a percentage rank from 0 to 1, where the highest height gets a rank close to 1
df.withColumn("%", percent_rank().over(percent)).show(50)

+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+--------------------+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|                Club|Height|Weight|Day|Month|Year|     Birth|                   %|
+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+--------------------+
|Argentina|     6|      DF|    FAZIO Federico|     FAZIO|       AS Roma (ITA)|   199|    85| 17|    3|1987|1987-03-17|                 0.0|
|Argentina|     1|      GK|     GUZMAN Nahuel|    GUZMÁN|   Tigres UANL (MEX)|   192|    90| 10|    2|1986|1986-02-10|0.045454545454545456|
|Argentina|    16|      DF|       ROJO Marcos|      ROJO|Manchester United...|   189|    82| 20|    3|1990|1990-03-20| 0.09090909090909091|
|Argentina|    12|      GK|     ARMANI Franco|    ARMANI|CA River Plate (ARG)|   189|    85| 16|   10|1986|1986-10-16| 0.09090909090909091|
|Argentina|    23|  

### Window Function 5 - Division into 'N' Parts: `ntile()`

In [65]:
# Define the window specification
# - partitionBy(col("Team")): partition the data by the "Team" column, so each team will have its own separate window
# - orderBy(desc(col("Height"))): order the rows within each partition by the "Height" column in descending order
tile = Window.partitionBy(col("Team")).orderBy(desc(col("Height")))

# Add a new column "Tile" to the DataFrame
# - ntile(5): divides the data within each partition into 5 approximately equal-sized parts (tiles)
# - over(tile): applies the window specification defined above to compute the tile number
# This means that within each team, rows will be divided into 5 groups, where each group (tile) contains roughly the same number of rows
df.withColumn("Tile", ntile(5).over(tile)).show(50)

+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+----+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|                Club|Height|Weight|Day|Month|Year|     Birth|Tile|
+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+----+
|Argentina|     6|      DF|    FAZIO Federico|     FAZIO|       AS Roma (ITA)|   199|    85| 17|    3|1987|1987-03-17|   1|
|Argentina|     1|      GK|     GUZMAN Nahuel|    GUZMÁN|   Tigres UANL (MEX)|   192|    90| 10|    2|1986|1986-02-10|   1|
|Argentina|    16|      DF|       ROJO Marcos|      ROJO|Manchester United...|   189|    82| 20|    3|1990|1990-03-20|   1|
|Argentina|    12|      GK|     ARMANI Franco|    ARMANI|CA River Plate (ARG)|   189|    85| 16|   10|1986|1986-10-16|   1|
|Argentina|    23|      GK|CABALLERO Wilfredo| CABALLERO|    Chelsea FC (ENG)|   186|    80| 28|    9|1981|1981-09-28|   1|
|Argenti

## Window Analytic Functions

- **Window Function 6 - LAG / Offset: `lag()`**
  - Retrieves the value from a previous row in the same column, offset by a specified number of rows within a window partition.

- **Window Function 7 - Lead / Offset: `lead()`**
  - Retrieves the value from a subsequent row in the same column, offset by a specified number of rows within a window partition.

### Window Function 6 - LAG / Offset: `lag()`

In [66]:
# Define the window specification
# - partitionBy(col("Team")): partition the data by the "Team" column, so each team will have its own separate window
# - orderBy(desc(col("Height"))): order the rows within each partition by the "Height" column in descending order
l = Window.partitionBy(col("Team")).orderBy(desc(col("Height")))

# Add a new column "Lag" to the DataFrame
# - lag(col("Height"), 2): shifts the value of the "Height" column by 2 rows within the specified window
# - over(l): applies the window specification defined above to compute the lagged values
# This means that within each team, the "Lag" column will show the height value from 2 rows earlier, relative to the current row's position
df.withColumn("Lag", lag(col("Height"), 2).over(l)).show(50)

+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+----+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|                Club|Height|Weight|Day|Month|Year|     Birth| Lag|
+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+----+
|Argentina|     6|      DF|    FAZIO Federico|     FAZIO|       AS Roma (ITA)|   199|    85| 17|    3|1987|1987-03-17|NULL|
|Argentina|     1|      GK|     GUZMAN Nahuel|    GUZMÁN|   Tigres UANL (MEX)|   192|    90| 10|    2|1986|1986-02-10|NULL|
|Argentina|    16|      DF|       ROJO Marcos|      ROJO|Manchester United...|   189|    82| 20|    3|1990|1990-03-20| 199|
|Argentina|    12|      GK|     ARMANI Franco|    ARMANI|CA River Plate (ARG)|   189|    85| 16|   10|1986|1986-10-16| 192|
|Argentina|    23|      GK|CABALLERO Wilfredo| CABALLERO|    Chelsea FC (ENG)|   186|    80| 28|    9|1981|1981-09-28| 189|
|Argenti

### Window Function 7 - Lead / Offset: `lead()`

In [67]:
# Define the window specification
# - partitionBy(col("Team")): partition the data by the "Team" column, so each team will have its own separate window
# - orderBy(desc(col("Height"))): order the rows within each partition by the "Height" column in descending order
l = Window.partitionBy(col("Team")).orderBy(desc(col("Height")))

# Add a new column "Lead" to the DataFrame
# - lead(col("Height")): shifts the value of the "Height" column by 1 row within the specified window
# - over(l): applies the window specification defined above to compute the lead values
# This means that within each team, the "Lead" column will show the height value from 1 row ahead, relative to the current row's position
df.withColumn("Lead", lead(col("Height")).over(l)).show(50)

+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+----+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|                Club|Height|Weight|Day|Month|Year|     Birth|Lead|
+---------+------+--------+------------------+----------+--------------------+------+------+---+-----+----+----------+----+
|Argentina|     6|      DF|    FAZIO Federico|     FAZIO|       AS Roma (ITA)|   199|    85| 17|    3|1987|1987-03-17| 192|
|Argentina|     1|      GK|     GUZMAN Nahuel|    GUZMÁN|   Tigres UANL (MEX)|   192|    90| 10|    2|1986|1986-02-10| 189|
|Argentina|    16|      DF|       ROJO Marcos|      ROJO|Manchester United...|   189|    82| 20|    3|1990|1990-03-20| 189|
|Argentina|    12|      GK|     ARMANI Franco|    ARMANI|CA River Plate (ARG)|   189|    85| 16|   10|1986|1986-10-16| 186|
|Argentina|    23|      GK|CABALLERO Wilfredo| CABALLERO|    Chelsea FC (ENG)|   186|    80| 28|    9|1981|1981-09-28| 184|
|Argenti

## Aggregations

### GroupBy + AGG 1

In [68]:
# Group by the "Team" column and calculate the average of the "Height" column
# - groupBy(col("Team")): groups the data by the "Team" column, creating a separate group for each team
# - agg({"Height": "avg"}): computes the average of the "Height" column for each group. The result will be a new column named "avg(Height)"
# - orderBy(col("avg(Height)"), ascending=False): sorts the groups by the average height in descending order
#   This ensures that teams with higher average heights appear first in the result
df.groupBy(col("Team")).agg({"Height": "avg"}) \
   .orderBy(col("avg(Height)"), ascending=False) \
   .show(50)

+--------------+------------------+
|          Team|       avg(Height)|
+--------------+------------------+
|        Serbia|186.69565217391303|
|       Denmark| 186.6086956521739|
|       Germany| 185.7826086956522|
|        Sweden| 185.7391304347826|
|       Iceland|185.52173913043478|
|       Belgium|185.34782608695653|
|       Croatia| 185.2608695652174|
|       Nigeria|184.52173913043478|
|       IR Iran|184.47826086956522|
|        Russia| 184.3913043478261|
|       Senegal|183.65217391304347|
|        France|183.30434782608697|
|        Poland|183.17391304347825|
|       Tunisia|183.08695652173913|
|   Switzerland|182.91304347826087|
|       England| 182.7391304347826|
|       Morocco|182.69565217391303|
|        Panama|182.17391304347825|
|Korea Republic| 181.8695652173913|
|       Uruguay|181.04347826086956|
|         Egypt|             181.0|
|     Australia| 180.8695652173913|
|        Brazil| 180.7826086956522|
|      Colombia| 180.7826086956522|
|    Costa Rica|180.69565217

### GroupBy + AGG 2

In [69]:
# Group by the "Team" column and calculate the average of the "Height" column
# - groupBy(col("Team")): groups the data by the "Team" column, creating a separate group for each team
# - agg(avg(col("Height")).alias("avg_Height")): computes the average of the "Height" column for each group
#   - avg(col("Height")): calculates the average height within each team
#   - alias("avg_Height"): renames the resulting average column to "avg_Height" for clarity
# - orderBy(col("avg_Height"), ascending=False): sorts the groups by the average height in descending order
#   This ensures that teams with higher average heights appear first in the result
df.groupBy(col("Team")).agg(avg(col("Height")).alias("avg_Height")) \
   .orderBy(col("avg_Height"), ascending=False) \
   .show(50)

+--------------+------------------+
|          Team|        avg_Height|
+--------------+------------------+
|        Serbia|186.69565217391303|
|       Denmark| 186.6086956521739|
|       Germany| 185.7826086956522|
|        Sweden| 185.7391304347826|
|       Iceland|185.52173913043478|
|       Belgium|185.34782608695653|
|       Croatia| 185.2608695652174|
|       Nigeria|184.52173913043478|
|       IR Iran|184.47826086956522|
|        Russia| 184.3913043478261|
|       Senegal|183.65217391304347|
|        France|183.30434782608697|
|        Poland|183.17391304347825|
|       Tunisia|183.08695652173913|
|   Switzerland|182.91304347826087|
|       England| 182.7391304347826|
|       Morocco|182.69565217391303|
|        Panama|182.17391304347825|
|Korea Republic| 181.8695652173913|
|       Uruguay|181.04347826086956|
|         Egypt|             181.0|
|     Australia| 180.8695652173913|
|        Brazil| 180.7826086956522|
|      Colombia| 180.7826086956522|
|    Costa Rica|180.69565217

## Where (Filter)

In [70]:
# Filter the DataFrame to include only rows where the "Team" column is "Brazil"
# - filter(col("Team") == "Brazil"): selects rows where the value in the "Team" column is equal to "Brazil"
# - show(25): displays the first 25 rows of the filtered DataFrame
df.filter(col("Team") == "Brazil").show(25)

+------+------+--------+-----------------+-----------+--------------------+------+------+---+-----+----+----------+
|  Team|Number|Position|FIFA Popular Name| Shirt Name|                Club|Height|Weight|Day|Month|Year|     Birth|
+------+------+--------+-----------------+-----------+--------------------+------+------+---+-----+----+----------+
|Brazil|    18|      MF|             FRED|       FRED|FC Shakhtar Donet...|   169|    64|  5|    3|1993|1993-03-05|
|Brazil|    21|      FW|           TAISON|     TAISON|FC Shakhtar Donet...|   172|    64| 13|    1|1988|1988-01-13|
|Brazil|    17|      MF|      FERNANDINHO|FERNANDINHO|Manchester City F...|   179|    67|  4|    5|1985|1985-05-04|
|Brazil|    22|      DF|           FAGNER|     FAGNER|SC Corinthians (BRA)|   168|    67| 11|    6|1989|1989-06-11|
|Brazil|    10|      FW|           NEYMAR|  NEYMAR JR|Paris Saint-Germa...|   175|    68|  5|    2|1992|1992-02-05|
|Brazil|    11|      MF|PHILIPPE COUTINHO|P. COUTINHO|  FC Barcelona (ES

In [71]:
# Filter the DataFrame to include only rows where the "Team" column is "Brazil"
# - where(col("Team") == "Brazil"): selects rows where the value in the "Team" column is equal to "Brazil"
# - show(25): displays the first 25 rows of the filtered DataFrame
df.where(col("Team") == "Brazil").show(25)

+------+------+--------+-----------------+-----------+--------------------+------+------+---+-----+----+----------+
|  Team|Number|Position|FIFA Popular Name| Shirt Name|                Club|Height|Weight|Day|Month|Year|     Birth|
+------+------+--------+-----------------+-----------+--------------------+------+------+---+-----+----+----------+
|Brazil|    18|      MF|             FRED|       FRED|FC Shakhtar Donet...|   169|    64|  5|    3|1993|1993-03-05|
|Brazil|    21|      FW|           TAISON|     TAISON|FC Shakhtar Donet...|   172|    64| 13|    1|1988|1988-01-13|
|Brazil|    17|      MF|      FERNANDINHO|FERNANDINHO|Manchester City F...|   179|    67|  4|    5|1985|1985-05-04|
|Brazil|    22|      DF|           FAGNER|     FAGNER|SC Corinthians (BRA)|   168|    67| 11|    6|1989|1989-06-11|
|Brazil|    10|      FW|           NEYMAR|  NEYMAR JR|Paris Saint-Germa...|   175|    68|  5|    2|1992|1992-02-05|
|Brazil|    11|      MF|PHILIPPE COUTINHO|P. COUTINHO|  FC Barcelona (ES

In [72]:
# Define the window specification
# - partitionBy(col("Team")): partition the data by the "Team" column, so each team will have its own separate window
# - orderBy(desc(col("Height"))): order the rows within each partition by the "Height" column in descending order
top1 = Window.partitionBy(col("Team")).orderBy(desc(col("Height")))

# Add a new column "Top" with ranking values for each row within the partition
# - rank().over(top1): computes the rank of each row within its partition based on the specified window
# - where(col("Top") == 1): filters the DataFrame to include only the rows where the rank ("Top") is 1
# - show(50): displays the first 50 rows of the filtered DataFrame
df.withColumn("Top", rank().over(top1)).where(col("Top") == 1).show(50)

+--------------+------+--------+-------------------+-------------+--------------------+------+------+---+-----+----+----------+---+
|          Team|Number|Position|  FIFA Popular Name|   Shirt Name|                Club|Height|Weight|Day|Month|Year|     Birth|Top|
+--------------+------+--------+-------------------+-------------+--------------------+------+------+---+-----+----+----------+---+
|     Argentina|     6|      DF|     FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|    85| 17|    3|1987|1987-03-17|  1|
|     Australia|    12|      GK|         JONES Brad|        JONES|Feyenoord Rotterd...|   193|    87| 19|    3|1982|1982-03-19|  1|
|       Belgium|     1|      GK|   COURTOIS Thibaut|     COURTOIS|    Chelsea FC (ENG)|   199|    91| 11|    5|1992|1992-05-11|  1|
|        Brazil|    16|      GK|             CASSIO|       CASSIO|SC Corinthians (BRA)|   195|    92|  6|    6|1987|1987-06-06|  1|
|      Colombia|    13|      DF|         MINA Yerry|      Y. MINA|  FC Barce

## Describe

In [73]:
# Display summary statistics of the DataFrame for numerical columns
# - describe(): computes summary statistics for numerical columns in the DataFrame
# - show(): displays the computed statistics in a tabular format
df.describe().show()

+-------+---------+-----------------+--------+-----------------+----------+--------------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|     Team|           Number|Position|FIFA Popular Name|Shirt Name|                Club|           Height|           Weight|               Day|             Month|              Year|
+-------+---------+-----------------+--------+-----------------+----------+--------------------+-----------------+-----------------+------------------+------------------+------------------+
|  count|      736|              736|     736|              736|       736|                 736|              736|              736|               736|               736|               736|
|   mean|     NULL|             12.0|    NULL|             NULL|      NULL|                NULL|182.4076086956522|77.18885869565217|15.793478260869565|5.8790760869565215| 1990.110054347826|
| stddev|     NULL|6.637760461599851|    NULL|    

In [74]:
# Filter the DataFrame to include only rows where the "Team" column is "Brazil"
# - where(col("Team") == "Brazil"): selects rows where the value in the "Team" column is equal to "Brazil"
# - describe(): computes summary statistics for numerical columns in the filtered DataFrame
# - show(): displays the computed statistics in a tabular format
df.where(col("Team") == "Brazil").describe().show()

+-------+------+-----------------+--------+-----------------+----------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|  Team|           Number|Position|FIFA Popular Name|Shirt Name|                Club|           Height|           Weight|              Day|            Month|             Year|
+-------+------+-----------------+--------+-----------------+----------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|    23|               23|      23|               23|        23|                  23|               23|               23|               23|               23|               23|
|   mean|  NULL|             12.0|    NULL|             NULL|      NULL|                NULL|180.7826086956522|76.56521739130434|11.26086956521739|6.130434782608695|1989.391304347826|
| stddev|  NULL|6.782329983125267|    NULL|             NULL|      NULL|        

## Aggregation Function Using Window Function

In [75]:
# Define the window specifications
# - param: partitions the data by the "Team" column and orders rows within each partition by "Height" in descending order
# - param2: partitions the data by the "Team" column without any specific order
param = Window.partitionBy(col("Team")).orderBy(desc(col("Height")))
param2 = Window.partitionBy(col("Team"))

# Add columns with aggregated values and filter to include only the top-ranked rows
# - withColumn("lineX", row_number().over(param)): adds a unique sequential number for each row within the team partition, ordered by height
# - withColumn("mean", avg(col("Height")).over(param2)): adds the average height for each team
# - withColumn("max", max(col("Height")).over(param2)): adds the maximum height for each team
# - withColumn("min", min(col("Height")).over(param2)): adds the minimum height for each team
# - where(col("lineX") == 1): filters to include only the top-ranked row by height within each team
# - select(col("Team"), col("mean"), col("max"), col("min")): selects the specified columns for display
# - orderBy(col("mean"), ascending=False): orders teams by average height in descending order
# - show(50): displays the first 50 rows
df.withColumn("lineX", row_number().over(param)) \
   .withColumn("mean", avg(col("Height")).over(param2)) \
   .withColumn("max", max(col("Height")).over(param2)) \
   .withColumn("min", min(col("Height")).over(param2)) \
   .where(col("lineX") == 1) \
   .select(col("Team"), col("mean"), col("max"), col("min")) \
   .orderBy(col("mean"), ascending=False) \
   .show(50)

+--------------+------------------+---+---+
|          Team|              mean|max|min|
+--------------+------------------+---+---+
|        Serbia|186.69565217391303|195|169|
|       Denmark| 186.6086956521739|200|171|
|       Germany| 185.7826086956522|195|176|
|        Sweden| 185.7391304347826|198|177|
|       Iceland|185.52173913043478|198|170|
|       Belgium|185.34782608695653|199|169|
|       Croatia| 185.2608695652174|201|172|
|       Nigeria|184.52173913043478|197|172|
|       IR Iran|184.47826086956522|194|177|
|        Russia| 184.3913043478261|196|173|
|       Senegal|183.65217391304347|196|173|
|        France|183.30434782608697|197|168|
|        Poland|183.17391304347825|195|172|
|       Tunisia|183.08695652173913|192|170|
|   Switzerland|182.91304347826087|192|165|
|       England| 182.7391304347826|196|170|
|       Morocco|182.69565217391303|190|167|
|        Panama|182.17391304347825|197|165|
|Korea Republic| 181.8695652173913|197|170|
|       Uruguay|181.043478260869